# Datenmanipulation mit Pandas in Python

## DataFrames einlesen: CSV, Excel, URL und wichtige Parameter

Pandas kann fast jedes Datenformat einlesen. Die wichtigsten Funktionen sind:

| Funktion           | Typische Quelle                         |
|--------------------|-----------------------------------------|
| `pd.read_csv()`    | CSV, TSV, Textdateien                   |
| `pd.read_excel()`  | .xlsx, .xls                             |
| `pd.read_json()`   | JSON, APIs                              |
| `pd.read_html()`   | Tabellen aus Webseiten                  |
| direkt aus URL     | beinhaltet obige Funktionen, ohne es vorher herunter zu laden |

### CSV einlesen: `pd.read_csv()` mit den wichtigsten Parametern

Die wichtigsten Parameter der Funktion sind Folgende:

| Parameter         | Wirkung                                      | Typisches Problem ohne diesen Parameter      |
|-------------------|----------------------------------------------|-----------------------------------------------|
| `sep` oder `delimiter` | Trenner in der CSV (oft `;`, `|`, `\t`)  | Spalten werden falsch getrennt                |
| `encoding`        | Zeichenkodierung (`utf-8`, `latin1`, `cp1252`) | Umlaute oder Fehler                       |
| `decimal`         | Dezimaltrennzeichen (`,` statt `.`)          | 52.000,50 wird zu String                      |
| `thousands`       | Tausendertrennzeichen (`.` oder Leerzeichen) | Zahlen werden als String gelesen              |
| `parse_dates`     | Spalten automatisch als Datum parsen         | Datum bleibt String                           |
| `dtype`           | Datentypen explizit setzen                   | Pandas rät falsch (z.B. ID als float)         |
| `usecols`         | Nur bestimmte Spalten laden                  | Speicher und Zeit sparen                        |
| `nrows`           | Nur erste n Zeilen laden                     | Schnell reinschauen                           |


#### Beispiel: Deutsche CSV mit Semikolon, Komma als Dezimal und Umlauten einlesen

Bevor man mit echten Dateien arbeitet, erstelle ich ein kleines, aber realistisches Beispiel:

```python
import pandas as pd
import numpy as np

data = {
    'Name': ['Anna', 'Ben', 'Clara', 'David', 'Emma'],
    'Alter': [28, 34, 19, 42, 31],
    'Gehalt_EUR': ['52.000,50', '68.000,00', '45.000,75', '82.000,00', '59.000,25'],
    'Eintritt': ['2021-03-15', '2020-11-01', '2023-07-20', '2019-01-10', '2022-05-05'],
    'Abteilung': ['IT', 'Vertrieb', 'IT', 'Vertrieb', 'Marketing']
}
df_demo = pd.DataFrame(data)
df_demo.to_csv('mitarbeiter_demo.csv', index=False, sep=';', decimal=',')
df_demo.to_excel('mitarbeiter_demo.xlsx', index=False)

print("Demo-Dateien erstellt: mitarbeiter_demo.csv und mitarbeiter_demo.xlsx")
```

Die Datei enthält typisch deutsche „Problemfälle“:
- Gehalt mit Punkt als Tausender- und Komma als Dezimaltrennzeichen ( 52.000,50)
- Semikolon als Spaltentrenner
- Datum im Format YYYY-MM-DD
- Umlaute und Sonderzeichen

So kann man alle wichtigen Parameter von `pd.read_csv()` und `pd.read_excel()` direkt testen, ohne dass man erst eine externe Datei herunterladen muss.

```python
df = pd.read_csv(
    'mitarbeiter_demo.csv',
    sep=';',                    # Deutscher Standard: Semikolon
    encoding='utf-8',           # oder 'latin1' bei alten Excel-Exports
    decimal=',',                # 52.000,50: float 52000.50
    thousands='.',              # 52.000,50: korrekt erkannt
    parse_dates=['Eintritt'],   # Spalte als datetime
    dtype={'Name': 'string', 'Abteilung': 'category'},  # Speicher sparen
    usecols=['Name', 'Alter', 'Gehalt_EUR', 'Eintritt', 'Abteilung'],  # nur diese Spalten
    nrows=None                   # None = alle Zeilen (oder z.B. 100 zum Testen)
)

print("Erfolgreich eingelesen!")
df.info()
df.head()
```

### Excel-Dateien einlesen: `pd.read_excel()`

Mit `pd.read_excel()` kann man .xlsx- und .xls-Dateien direkt einlesen, ohne Excel öffnen zu müssen. Wichtige Parameter sind sheet_name (welches Tabellenblatt?), skiprows (Überschriften oder Metadaten überspringen), usecols (nur bestimmte Spalten oder Bereiche wie „A:F“ laden) und dtype, um z. B. Kundennummern als String statt als Zahl zu erzwingen. Pandas kann Excel-Dateien sogar direkt aus einer URL laden. 

```python
df_excel = pd.read_excel(
    'mitarbeiter_demo.xlsx',
    sheet_name='Sheet1',        # oder 0 für erstes Blatt
    usecols='A:E',              # Excel-Spaltenbereich
    skiprows=2,                 # z.B. Überschriften überspringen
    dtype={'Gehalt_EUR': str}  # falls Excel komische Formate hat
)
```

### URL einlesen: 

(Demnächst)

## Selektion & Indexing in Pandas

### Selektion mittels `.loc[]`, `.iloc[]` und `df[]`

Die Bibliothek Pandas bietet mehrere Wege, um Zeilen und Spalten eines DataFrames auszuwählen. Je nach Situation ist eine Methode besser als die Andere.

| Methode         | Was sie macht                         | Empfohlen für                     |
|-----------------|---------------------------------------|-----------------------------------|
| `df[]`          | Einfache Auswahl (Spalten oder bool)  | Schnell und einfach                 |
| `.loc[]`        | Labelbasiert (Namen der Spalten/Zeilen)| Immer dann, wenn man Namen verwendet  |
| `.iloc[]`       | Positionsbasiert (Indexnummer der Spalten/Zeilen)    | Wenn man mit Position arbeitet und es größere Dataframes sind   |


`.loc[]` und `.iloc[]` sind sicherer als `[]`, weil sie keine Verwirrung zwischen Label und Position zulassen. Die einfachen eckigen Klammern `df[]` sind zwar praktisch, können aber zu unerwartetem Verhalten führen, wenn sich Spaltennamen und Indexwerte überschneiden. `.loc[]` und `.iloc[]` hingegen sind explizit:  
- **`.loc[]`** arbeitet ausschließlich mit Spalten- und Zeilennamen,  
- **`.iloc[]`** ausschließlich mit numerischen Positionen (Indexen).  
  
Das Wichtigste bei `.loc[]` ist, dass das Ende des Slices immer inklusiv ist.

```python
df.loc['2020-01-01':'2020-12-31']    # Hier wird der 31. Dezember 2020 mit ausgegeben
df.loc['P2':'P5']                    # P2, P3, P4 und P5 werden angezeigt
df.loc[10:50]                        # wenn der Index aus Zahlen besteht, so sind 10 bis 50 inklusive
```

Die `.iloc[]` Funktion ist positionsbasiert und es zählt nur die numerische Position. Wichtig zu bemerken ist, genau wie bei Listen gilt, dass das Ende des Slices exklusiv (nicht mit eingeschlossen) ist.

```python
df.iloc[0:5]     # Zeilen an Position 0, 1, 2, 3, 4. Die 5. Zeile (Index 4) ist die letzte
df.iloc[1:4]     # Position 1, 2, 3
df.iloc[-3:]     # die letzten drei Zeilen 
```
 
#### Direkter Vergleich zwischen `.loc[]` und `.iloc[]`:

Angenommen, unser DataFrame hat den Index `['A', 'B', 'C', 'D', 'E']`:

```python
df.loc['B':'D']   # Diese Methode liefert B, C und D (D ist inklusiv)
df.iloc[1:4]      # liefert ebenfalls B, C, D (D ist exklusiv)

df.loc['B':'E']   # liefert B, C, D und E (E ist dabei)
df.iloc[1:5]      # liefert B, C, D und E (erst bei Position 5 wird E erreicht)
df.iloc[1:4]      # würde nur B, C, D liefern (E fehlt)
```

### Boolsche Masken und komplexe Bedingungen

Eine weitere mächtigste Funktionen von Pandas ist die Filterung mit booleschen Masken. Statt Schleifen zu schreiben, erzeugt man einen booleschen Vektor (True/False) und übergibt ihn an das DataFrame:

```python
df[df['Alter'] > 30]                                   # einfache Bedingung
df[(df['Stadt'] == 'Berlin') & (df['Gehalt'] > 60000)] # Konjunktion
df[(df['Abteilung'] == 'IT') | (df['Alter'] < 25)]     # Disjunktion
```

Wichtig zu bemerken ist, dass bei mehreren Bedingungen runde Klammern um jede einzelne Bedingung stehen müssen und die Operatoren `&` (und), `|` (oder) sowie `~` (nicht) werden verwendet.


## Neue Spalten erstellen & löschen
- Einfache Berechnungen und Zuweisungen
- apply() und lambda-Funktionen
- map() und replace()
- Bedingte Spalten mit np.where() und np.select()
- drop(), pop(), del

## Daten bereinigen (Cleaning)
- Umgang mit fehlenden Werten (isnull, dropna, fillna)
- Duplikate finden und entfernen
- Datentypen konvertieren (astype, pd.to_datetime, pd.to_numeric)
- String-Methoden (.str.) – lower, upper, contains, replace, split, strip

## Sortieren, Ranking & Reihenfolge
- sort_values() und sort_index()
- rank()

## Gruppieren & Aggregation

Die `groupby`-Funktion ist eines der mächtigsten Werkzeuge in Pandas. Sie ermöglicht es, Daten nach einer oder mehreren Spalten zu gruppieren und dann Aggregationen (Zusammenfassungen) über diese Gruppen durchzuführen. 
Beispieldokument

In [7]:
import pandas as pd
import numpy as np

data = {
    'Name': ['Anna', 'Ben', 'Clara', 'David', 'Emma'],
    'Alter': [28, 34, 19, 42, 31],
    'Gehalt_EUR': ['52.000,50', '68.000,00', '45.000,75', '82.000,00', '59.000,25'],
    'Eintritt': ['2021-03-15', '2020-11-01', '2023-07-20', '2019-01-10', '2022-05-05'],
    'Abteilung': ['IT', 'Vertrieb', 'IT', 'Vertrieb', 'Marketing']
}
df = pd.DataFrame(data)

df.to_csv('mitarbeiter_demo.csv', index=False, sep=';', decimal=',')
df = pd.read_csv(
    'mitarbeiter_demo.csv',
    sep=';', 
    encoding='utf-8', 
    decimal=',', 
    thousands='.', 
    parse_dates=['Eintritt'], 
    dtype={'Name': 'string', 'Abteilung': 'category'}
)

print(df.head())

    Name  Alter  Gehalt_EUR   Eintritt  Abteilung
0   Anna     28    52000.50 2021-03-15         IT
1    Ben     34    68000.00 2020-11-01   Vertrieb
2  Clara     19    45000.75 2023-07-20         IT
3  David     42    82000.00 2019-01-10   Vertrieb
4   Emma     31    59000.25 2022-05-05  Marketing


### Einfache Aggregationen (mean, sum, count, etc.)

Mit `groupby()` kann man Daten nach einer Kategorie gruppieren und dann eine einfache Aggregationsfunktion anwenden, wie `mean()` für den Durchschnitt, `sum()` für die Summe oder `count()` für die Anzahl. Die Funktion wird auf alle numerischen Spalten angewendet, es sei denn, man spezifiziert eine Spalte. Die `groupby('Spaltenname')` Funktion teilt den DataFrame in Gruppen nach der Spalte 'Spaltenname' auf. Dann wendet `mean()` den Durchschnitt auf die angegebene Spalte an. Der Parameter `observed=True` vermeidet Warnings bei kategorischen Spalten.


In [8]:
# Durchschnittsgehalt pro Abteilung
durchschnitt_gehalt = df.groupby('Abteilung', observed=True)['Gehalt_EUR'].mean()
print("Durchschnittsgehalt pro Abteilung:")
print(durchschnitt_gehalt)

# Summe der Alter pro Abteilung
summe_alter = df.groupby('Abteilung', observed=True)['Alter'].sum()
print("\nSumme der Alter pro Abteilung:")
print(summe_alter)

# Anzahl Mitarbeiter pro Abteilung
anzahl = df.groupby('Abteilung', observed=True).size()
print("\nAnzahl Mitarbeiter pro Abteilung:")
print(anzahl)

Durchschnittsgehalt pro Abteilung:
Abteilung
IT           48500.625
Marketing    59000.250
Vertrieb     75000.000
Name: Gehalt_EUR, dtype: float64

Summe der Alter pro Abteilung:
Abteilung
IT           47
Marketing    31
Vertrieb     76
Name: Alter, dtype: int64

Anzahl Mitarbeiter pro Abteilung:
Abteilung
IT           2
Marketing    1
Vertrieb     2
dtype: int64


- Mehrere Aggregationsfunktionen gleichzeitig (agg())
- named aggregation & as_index=False
- Gruppierte Filterungen


## Merges 
- concat() – Zeilen und Spalten zusammenhängen
- merge() – inner, left, right, outer
- join() auf Index
- Praxisbeispiel: Kundendaten + Bestelldaten verbinden

## Pivot-Tabellen
- pivot() und pivot_table()
- margins & margins_name
